# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_CBS_INHABITANTS_PER_MUNICIPALITY]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_CBS_INHABITANTS_PER_MUNICIPALITY]
    START WITH 1
    INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY]'))
CREATE TABLE [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY] (
    [ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_CBS_INHABITANTS_PER_MUNICIPALITY]),
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [REGIOS] [VARCHAR](100),
    [CODE_1] [VARCHAR](100),
	[CODE_42] [VARCHAR](100),
	[NAAM_43] [VARCHAR](100),
	[INWONERTAL_50] [VARCHAR](100)
);
GO

### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_CBS_INHABITANTS_PER_MUNICIPALITY]') AND TYPE = 'SO')
CREATE SEQUENCE [SEQ_CBS_INHABITANTS_PER_MUNICIPALITY]
    START WITH 1
    INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY]'))
CREATE TABLE [VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY] (
    [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_CBS_INHABITANTS_PER_MUNICIPALITY]),
    [VRCODE] VARCHAR(5),
    [GMCODE] VARCHAR(10),
	[INHABITANTS] BIGINT,
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

IF NOT EXISTS(SELECT * FROM SYS.INDEXES WHERE NAME='IX_VWS_VWSSTATIC_CBS_INHABITANTS_PER_MUNICIPALITY')
CREATE NONCLUSTERED INDEX [IX_VWS_VWSSTATIC_CBS_INHABITANTS_PER_MUNICIPALITY] ON [VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY] (
    [DATE_LAST_INSERTED], 
    [VRCODE], 
    [GMCODE]
) INCLUDE ([INHABITANTS]);
GO

# **STORED PROCEDURES**

---

In 2022 multiple municipalties (i.e. gemeentes) are reclassified (e.g. merging) into novel muncipalities. To implement these changes, several database adjustments had to be performed.

Within the `[dbo].[SP_INHABITANTS_PER_REGION_STATIC]`, with help of the mapping-tabel (i.e. `[RECLASSIFIED_MUNICIPALITY_MAPPING]` ), the reclassified muncipalities will be progated towards the `[VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY]`:

```sql
LEFT JOIN (
    SELECT 
        * 
    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
    WHERE [ACTIVE] = 1 
        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
) B ON A.[CODE_1]=B.[GM_CODE_ORIGINAL]
```

In turn, all downstream procedures, views or tabel - depending on `[VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY]` - will automatcally implement the modified muncipality codes (i.e. `GM_CODE`, `GMCODE` or `CODE_1`).

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [dbo].[SP_INHABITANTS_PER_REGION_STATIC]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[CBS_INHABITANTS_PER_MUNICIPALITY]  (
        [VRCODE], 
        [GMCODE], 
        [INHABITANTS]
    )
    SELECT
        ISNULL(B.[VR_CODE], [CODE_42]),
        ISNULL(B.[GM_CODE_NEW], [CODE_1]),
        CAST(ROUND(SUM(CAST(A.[INWONERTAL_50] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS INT)
    FROM [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY] A
        LEFT JOIN   (
                    SELECT 
                        * 
                    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
                    WHERE [ACTIVE] = 1 
                        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
                ) B ON A.[CODE_1] = B.[GM_CODE_ORIGINAL]
    WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSSTAGE].[CBS_INHABITANTS_PER_MUNICIPALITY])
    GROUP BY ISNULL(B.[VR_CODE],[CODE_42]), ISNULL(B.[GM_CODE_NEW],[CODE_1])
END;
GO

# **DATATINO CONFIGURATIONS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'CBS_DATA_MONTHLY',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

-- 2) UPSERT WORKFLOW(S).....
SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @SCHEDULE = '0 13 1-7 * FRI', -- AT 01:00 PM, BETWEEN DAY 1 AND 7 OF THE MONTH, AND ON FRIDAY
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'https://opendata.cbs.nl/ODataApi/odata/84721NED/TypedDataSet?$select=RegioS,%20Code_1,%20Code_42,%20Naam_43,%20Inwonertal_52'
    WHEN 'acceptance' THEN 'https://opendata.cbs.nl/ODataApi/odata/84721NED/TypedDataSet?$select=RegioS,%20Code_1,%20Code_42,%20Naam_43,%20Inwonertal_52'
    ELSE 'datafiles/STATIC/cbs_inhabitants_per_municipality.json'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'Web'
    WHEN 'acceptance' THEN 'Web'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = 'SOURCE_LOAD_VWSSTAGE_CBS_INHABITANTS_DATA';
SET @source_description = CONCAT('LOAD: CBS_INHABITANTS_PER_MUNICIPALITY - ', @workflow_name);
SET @target_name = 'VWSSTAGE.CBS_INHABITANTS_PER_MUNICIPALITY';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'RegioS|Code_1|Code_42|Naam_43|Inwonertal_52',
    @target_columns = 'REGIOS|CODE_1|CODE_42|NAAM_43|INWONERTAL_50|DATE_LAST_INSERTED=GETDATE',
    @target_name = @target_name,
    @source_type = 'JsonFile',
    @location_type = @location_type,
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO STATIC......
SET @source = 'dbo.SP_INHABITANTS_PER_REGION_STATIC';
SET @source_name = 'SOURCE_Static_Inhabitants_data';
SET @source_description = 'MAP: CBS_INHABITANTS_PER_MUNICIPALITY FROM STAGE TO STATIC';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = 'PROCESS_LOAD_VWSSTAGE_CBS_INHABITANTS_DATA';
SET @process_description = CONCAT('LOAD: CBS_INHABITANTS_PER_MUNICIPALITY - ', @workflow_name);
SET @process_source_name = 'SOURCE_LOAD_VWSSTAGE_CBS_INHABITANTS_DATA';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO STATIC......
SET @process_name = 'PROCESS_Static_Inhabitants_data';
SET @process_description = 'MAP: CBS_INHABITANTS_PER_MUNICIPALITY FROM STAGE TO STATIC';
SET @process_source_name = 'SOURCE_Static_Inhabitants_data';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = 'DEPENDENCY_PROCESS_Static_Inhabitants_data';
SET @dependency_dataflow_name = 'PROCESS_Static_Inhabitants_data';
SET @dependency_process_name = 'PROCESS_LOAD_VWSSTAGE_CBS_INHABITANTS_DATA';
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO